# 📊 Demo Completo: Structured Streaming com Parquet

Notebook completo para demonstrar **Structured Streaming** processando arquivos Parquet arquivo por arquivo.

## 🎯 Objetivos
- Processar arquivos um por um com `maxFilesPerTrigger=1`
- Gerar dados sintéticos de e-commerce (8 arquivos Parquet)
- Demonstrar processamento visível batch por batch
- Criar tabelas Delta Lake automaticamente
- Agregações em tempo real por categoria e região
- Visualizar métricas completas de processamento

## ✅ Compatível com Databricks Free Edition
- Sem dependências externas (Kafka, etc)
- Usa dados sintéticos gerados pelo próprio notebook
- Funciona em Serverless Compute

## 1️⃣ Setup e Configuração

In [ ]:
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime, timedelta
import random

# Detectar ambiente
IS_DATABRICKS = 'DATABRICKS_RUNTIME_VERSION' in os.environ

# Configurar paths
if IS_DATABRICKS:
    BASE_PATH = '/FileStore/streaming_demo'
else:
    BASE_PATH = '/tmp/streaming_demo'

INPUT_PATH = f'{BASE_PATH}/input'
OUTPUT_PATH = f'{BASE_PATH}/output'
CHECKPOINT_PATH = f'{BASE_PATH}/checkpoint'

print(f'🔧 Ambiente: {"Databricks" if IS_DATABRICKS else "Local"}')
print(f'📂 Input: {INPUT_PATH}')
print(f'💾 Output: {OUTPUT_PATH}')
print(f'🔖 Checkpoint: {CHECKPOINT_PATH}')

## 2️⃣ Gerar Dados Sintéticos de E-commerce

In [ ]:
# Schema dos dados
schema = StructType([
    StructField('transaction_id', IntegerType(), False),
    StructField('customer_id', IntegerType(), False),
    StructField('product', StringType(), False),
    StructField('category', StringType(), False),
    StructField('quantity', IntegerType(), False),
    StructField('price', DoubleType(), False),
    StructField('total', DoubleType(), False),
    StructField('timestamp', TimestampType(), False),
    StructField('region', StringType(), False),
    StructField('payment_method', StringType(), False)
])

def generate_transactions(n=500, batch_id=0):
    """Gera transações sintéticas de e-commerce"""
    categories = ['Electronics', 'Books', 'Clothing', 'Food', 'Sports', 'Home']
    products = {
        'Electronics': ['Laptop', 'Phone', 'Tablet', 'Headphones'],
        'Books': ['Fiction', 'Tech', 'Business', 'Science'],
        'Clothing': ['Shirt', 'Pants', 'Shoes', 'Jacket'],
        'Food': ['Snacks', 'Drinks', 'Frozen', 'Fresh'],
        'Sports': ['Equipment', 'Apparel', 'Accessories'],
        'Home': ['Furniture', 'Decor', 'Kitchen', 'Garden']
    }
    regions = ['North', 'South', 'East', 'West', 'Central']
    payments = ['Credit Card', 'Debit Card', 'Cash', 'Digital Wallet']
    
    data = []
    base_id = batch_id * n
    base_time = datetime.now() - timedelta(hours=24)
    
    for i in range(n):
        category = random.choice(categories)
        product = random.choice(products[category])
        qty = random.randint(1, 5)
        price = round(random.uniform(10, 500), 2)
        
        data.append((
            base_id + i,
            random.randint(1000, 9999),
            product,
            category,
            qty,
            price,
            round(qty * price, 2),
            base_time + timedelta(minutes=random.randint(0, 1440)),
            random.choice(regions),
            random.choice(payments)
        ))
    
    return data

print('✅ Função de geração definida!')

In [ ]:
print('🎲 Gerando 8 arquivos Parquet para demonstração...\n')

# Limpar pasta de input
if IS_DATABRICKS:
    try:
        dbutils.fs.rm(INPUT_PATH, True)
    except:
        pass
    dbutils.fs.mkdirs(INPUT_PATH)

# Gerar 8 arquivos
for i in range(8):
    data = generate_transactions(n=300, batch_id=i)
    df = spark.createDataFrame(data, schema)
    
    file_path = f'{INPUT_PATH}/transactions_batch_{i:03d}.parquet'
    df.coalesce(1).write.mode('overwrite').parquet(file_path)
    
    print(f'  ✅ Arquivo {i+1}/8: transactions_batch_{i:03d}.parquet ({len(data)} transações)')

print(f'\n✅ Total: {8 * 300} transações em 8 arquivos')
print(f'📂 Salvos em: {INPUT_PATH}')

## 3️⃣ Configurar Structured Streaming com maxFilesPerTrigger=1

In [ ]:
# Criar streaming DataFrame com maxFilesPerTrigger=1
streaming_df = spark.readStream \
    .schema(schema) \
    .option('maxFilesPerTrigger', 1) \
    .parquet(INPUT_PATH)

# Adicionar colunas de controle
enriched_df = streaming_df \
    .withColumn('processing_time', current_timestamp()) \
    .withColumn('file_name', input_file_name()) \
    .withColumn('date', to_date(col('timestamp'))) \
    .withColumn('hour', hour(col('timestamp'))) \
    .withColumn('revenue', round(col('total'), 2))

print('✅ Streaming DataFrame configurado!')
print(f'\n📊 Schema:')
enriched_df.printSchema()
print(f'\n🔄 É streaming? {streaming_df.isStreaming}')
print(f'⚙️  maxFilesPerTrigger: 1 (processa 1 arquivo por vez)')
print(f'📝 Colunas adicionadas: processing_time, file_name, date, hour, revenue')

## 4️⃣ Criar Agregações por Categoria e Região

In [ ]:
# Agregação por categoria e região
aggregated_df = enriched_df \
    .groupBy('category', 'region', 'date') \
    .agg(
        count('*').alias('total_transactions'),
        sum('quantity').alias('total_items'),
        sum('revenue').alias('total_revenue'),
        avg('revenue').alias('avg_transaction_value'),
        countDistinct('customer_id').alias('unique_customers'),
        min('timestamp').alias('first_transaction'),
        max('timestamp').alias('last_transaction')
    ) \
    .withColumn('avg_transaction_value', round(col('avg_transaction_value'), 2)) \
    .withColumn('total_revenue', round(col('total_revenue'), 2))

print('✅ Pipeline de agregação criado!')
print('📊 Métricas calculadas:')
print('   • Total de transações')
print('   • Total de itens vendidos')
print('   • Receita total')
print('   • Valor médio por transação')
print('   • Clientes únicos')
print('   • Primeira e última transação')

## 5️⃣ Definir Função de Processamento Batch (foreachBatch)

In [ ]:
# Estatísticas globais
batch_stats = {'count': 0, 'total_records': 0, 'total_revenue': 0}

def process_batch(batch_df, batch_id):
    """Processa cada batch mostrando métricas detalhadas"""
    batch_stats['count'] += 1
    
    print('\n' + '='*100)
    print(f'📦 BATCH #{batch_stats["count"]} | Batch ID: {batch_id}')
    print('='*100)
    
    num_records = batch_df.count()
    
    if num_records == 0:
        print('⚠️  Batch vazio - aguardando novos arquivos...')
        return
    
    batch_stats['total_records'] += num_records
    
    # Informações do arquivo processado
    files = batch_df.select('file_name').distinct().collect()
    if files:
        print(f'\n📄 Arquivo(s) processado(s):')
        for f in files:
            fname = f.file_name.split('/')[-1] if f.file_name else 'unknown'
            print(f'   • {fname}')
    
    # Estatísticas do batch
    stats = batch_df.agg(
        count('*').alias('transactions'),
        sum('revenue').alias('revenue'),
        countDistinct('customer_id').alias('customers'),
        avg('revenue').alias('avg_ticket')
    ).collect()[0]
    
    batch_stats['total_revenue'] += stats.revenue if stats.revenue else 0
    
    print(f'\n📊 ESTATÍSTICAS DO BATCH:')
    print(f'   💳 Transações: {stats.transactions:,}')
    print(f'   💰 Receita: ${stats.revenue:,.2f}')
    print(f'   👥 Clientes: {stats.customers:,}')
    print(f'   🎯 Ticket Médio: ${stats.avg_ticket:,.2f}')
    
    # Top categorias
    print(f'\n🏆 TOP 5 CATEGORIAS (Receita):')
    top_cat = batch_df.groupBy('category') \
        .agg(sum('revenue').alias('revenue')) \
        .orderBy(col('revenue').desc()) \
        .limit(5)
    top_cat.show(truncate=False)
    
    # Distribuição regional
    print(f'🌎 DISTRIBUIÇÃO POR REGIÃO:')
    regions = batch_df.groupBy('region') \
        .agg(
            count('*').alias('transactions'),
            sum('revenue').alias('revenue')
        ) \
        .orderBy(col('revenue').desc())
    regions.show(truncate=False)
    
    # Totais acumulados
    print(f'\n📈 TOTAIS ACUMULADOS:')
    print(f'   📦 Batches processados: {batch_stats["count"]}')
    print(f'   📊 Transações totais: {batch_stats["total_records"]:,}')
    print(f'   💵 Receita total: ${batch_stats["total_revenue"]:,.2f}')
    
    print('\n' + '-'*100)
    print(f'✅ Batch #{batch_stats["count"]} concluído!')
    print('-'*100 + '\n')

print('✅ Função process_batch() definida!')

## 6️⃣ EXECUTAR Streaming com foreachBatch

**Execute esta célula e observe o processamento arquivo por arquivo!**

In [ ]:
# Resetar estatísticas
batch_stats = {'count': 0, 'total_records': 0, 'total_revenue': 0}

print('🚀 INICIANDO STRUCTURED STREAMING')
print('='*100)
print(f'📂 Source: {INPUT_PATH}')
print(f'⚙️  maxFilesPerTrigger: 1')
print(f'⏱️  Trigger: a cada 5 segundos')
print(f'🔖 Checkpoint: {CHECKPOINT_PATH}')
print('='*100)

# Iniciar stream com foreachBatch
query = enriched_df.writeStream \
    .foreachBatch(process_batch) \
    .trigger(processingTime='5 seconds') \
    .option('checkpointLocation', CHECKPOINT_PATH) \
    .start()

print(f'\n✅ Stream ativo!')
print(f'🆔 Query ID: {query.id}')
print(f'📊 Status: {query.status}')
print(f'\n💡 Observe o processamento arquivo por arquivo abaixo...')
print(f'⏸️  Execute a próxima célula para parar após ~60 segundos')

## 7️⃣ Parar o Streaming

**Aguarde ~60 segundos para processar todos os arquivos, então execute esta célula**

In [ ]:
import time

print('⏳ Aguardando 60 segundos para processar todos os arquivos...')
time.sleep(60)

query.stop()

print('\n' + '='*100)
print('🛑 STREAMING FINALIZADO')
print('='*100)
print(f'\n📊 RESUMO FINAL:')
print(f'   📦 Total de batches: {batch_stats["count"]}')
print(f'   💳 Total de transações: {batch_stats["total_records"]:,}')
print(f'   💰 Receita total: ${batch_stats["total_revenue"]:,.2f}')
print(f'   ✅ Checkpoint salvo em: {CHECKPOINT_PATH}')
print('='*100)

## 8️⃣ OPÇÃO 2: Gravar em Tabela Delta Lake

**Alternativa: processar e gravar diretamente em Delta Lake**

In [ ]:
print('🚀 Iniciando streaming com saída para Delta Lake...')
print(f'💾 Tabela: {OUTPUT_PATH}')
print(f'🔖 Checkpoint: {CHECKPOINT_PATH}_delta\n')

# Streaming para Delta com agregações
query_delta = aggregated_df.writeStream \
    .format('delta') \
    .outputMode('complete') \
    .option('checkpointLocation', f'{CHECKPOINT_PATH}_delta') \
    .trigger(processingTime='5 seconds') \
    .start(OUTPUT_PATH)

print(f'✅ Stream Delta ativo! Query ID: {query_delta.id}')
print(f'💡 Execute as próximas células após ~60 segundos')

In [ ]:
import time
time.sleep(60)
query_delta.stop()
print('✅ Stream Delta finalizado!')

## 9️⃣ Consultar Tabela Delta

In [ ]:
# Ler tabela Delta
delta_df = spark.read.format('delta').load(OUTPUT_PATH)

print(f'📊 Tabela Delta carregada!')
print(f'📈 Total de registros: {delta_df.count():,}\n')

print('🏆 TOP 10 - Categoria + Região (por Receita):')
delta_df.orderBy(col('total_revenue').desc()).limit(10).show(truncate=False)

# Totais gerais
totals = delta_df.agg(
    sum('total_transactions').alias('transactions'),
    sum('total_revenue').alias('revenue'),
    sum('unique_customers').alias('customers')
).collect()[0]

print(f'\n💰 TOTAIS GERAIS:')
print(f'   💳 Transações: {totals.transactions:,.0f}')
print(f'   💵 Receita: ${totals.revenue:,.2f}')
print(f'   👥 Clientes: {totals.customers:,.0f}')

## 🔟 Criar Tabela Gerenciada no Catalog (Databricks)

In [ ]:
if IS_DATABRICKS:
    # Criar database
    spark.sql('CREATE DATABASE IF NOT EXISTS streaming_demo')
    
    # Criar tabela gerenciada
    delta_df.write \
        .format('delta') \
        .mode('overwrite') \
        .saveAsTable('streaming_demo.sales_summary')
    
    print('✅ Tabela criada: streaming_demo.sales_summary')
    print('\n📊 Amostra da tabela:')
    
    spark.sql('SELECT * FROM streaming_demo.sales_summary LIMIT 10').show(truncate=False)
    
    print('\n💡 Agora você pode consultar com SQL:')
    print('   SELECT category, SUM(total_revenue) FROM streaming_demo.sales_summary GROUP BY category')
else:
    print('⚠️ Tabelas gerenciadas disponíveis apenas no Databricks')

## 1️⃣1️⃣ Limpeza de Recursos

In [ ]:
print('🧹 Limpando recursos...\n')

# Parar todos os streams ativos
for s in spark.streams.active:
    s.stop()
    print(f'✓ Stream {s.id} parado')

# Remover arquivos (Databricks)
if IS_DATABRICKS:
    paths = [INPUT_PATH, OUTPUT_PATH, CHECKPOINT_PATH, f'{CHECKPOINT_PATH}_delta']
    for path in paths:
        try:
            dbutils.fs.rm(path, True)
            print(f'✓ Removido: {path}')
        except:
            pass

print('\n✅ Limpeza concluída!')

---

## 📚 Resumo do Notebook

### O que foi demonstrado:
1. ✅ **Structured Streaming** com `maxFilesPerTrigger=1`
2. ✅ Processamento arquivo por arquivo visível em tempo real
3. ✅ Agregações dinâmicas por categoria e região
4. ✅ Gravação em **Delta Lake**
5. ✅ Criação de **tabelas gerenciadas** no Catalog
6. ✅ Métricas detalhadas: transações, receita, clientes, ticket médio
7. ✅ Monitoramento de streams ativos

### Conceitos-chave para ensinar:
- **maxFilesPerTrigger**: controla quantos arquivos processar por trigger
- **foreachBatch**: permite lógica customizada para cada micro-batch
- **checkpointLocation**: garante exactly-once processing e recuperação de falhas
- **Delta Lake**: formato ACID transacional para data lakes
- **outputMode**: 
  - `append` - adiciona novos registros
  - `complete` - reescreve resultado completo (usado com agregações)
  - `update` - atualiza apenas registros modificados

### Configurações importantes:
- **trigger**: define quando processar (processingTime, once, continuous)
- **Schema inference**: sempre defina schema explícito em produção
- **Watermarks**: para processar dados com atraso temporal
- **Particionamento**: organize dados por data, região, categoria

### Para produção:
- Ajustar `maxFilesPerTrigger` conforme volume de dados
- Configurar triggers apropriados (não usar 5 segundos em prod)
- Implementar watermarks para dados atrasados
- Adicionar tratamento de erros robusto
- Configurar alertas e monitoramento (Spark UI, Ganglia, Grafana)
- Usar particionamento eficiente no Delta Lake
- Testar recuperação de falhas (checkpoint)

### Próximos passos para seus alunos:
1. Modificar agregações (adicionar mais métricas)
2. Experimentar diferentes outputModes
3. Adicionar watermarks para janelas temporais
4. Testar com volumes maiores de dados
5. Integrar com fontes reais (S3, Azure Blob, GCS)

**🎓 Perfeito para ensinar os fundamentos de Structured Streaming!**